In [70]:
from pymongo import MongoClient
from BeautifulSoup import BeautifulSoup
from urlparse import urljoin
import urllib2
import argparse
import re

ImportError: No module named pymongo

In [40]:
search_term = 'Stipper Club'
city = 'Portland OR'
client = MongoClient()
db = client[search_term]
collection = db[city]

YELP_URL = 'https://www.yelp.com' 

In [51]:
def crawl_results(search_term, city, database=None):
    yelp_search_url = '{}/search?find_desc={}s&find_loc={}'.format(YELP_URL, search_term.replace(' ', '+'),
                                                                   city.replace(' ', '+'))
    soup = BeautifulSoup(urllib2.urlopen(yelp_search_url).read())
    header = soup.find('h1').text
    print("Scraping page: {}".format(header))
    # grab list of results, iterate through list and then find the next page
    results = soup.findAll('div', attrs={'class': re.compile(r'media-story')})
    # crawl results and pull of the results
    # if there is a 'next' page go to it and then 
    for r in results:
        sub_href = r.find('a')
        if sub_href:
            if sub_href['href'].startswith('/biz/'):
                result_page = '{}{}'.format(YELP_URL, sub_href['href'])
                crawl_results_page_businesses(result_page, city, database)

In [42]:
def crawl_results_page_businesses(yelp_page_url, city, database=None):
    while True:
        soup = BeautifulSoup(urllib2.urlopen(yelp_page_url).read())
        header = soup.find('h1').text
        print("Scraping page: {}".format(header))
        # grab list of results, iterate through list and then find the next page
        results = soup.findAll('div', attrs={'class': re.compile(r'media-story')})
        # crawl results and pull of the results
        # if there is a 'next' page go to it and then 
        for r in results:
            sub_href = r.find('a')
            if sub_href:
                if sub_href['href'].startswith('/biz/'):
                    result_page = sub_href['href']
                    crawl_business_reviews(result_page, city, database=None)
        # find next button, if there is one
        next_page = soup.find('a', attrs={'class': 'u-decoration-none next pagination-links_anchor'})
        if next_page:
            yelp_page_url = next_page['href']
        else:
            break

In [43]:
def crawl_business_reviews(yelp_result_page, city, database=None):
    counter = 0
    while True:
        soup = BeautifulSoup(urllib2.urlopen(yelp_result_page).read())
        business_dict = {}
        business_dict['business-name'] = soup.find('h1').text
        business_dict['score'] = soup.find('div', attrs={'class':'biz-rating biz-rating-very-large clearfix'}).img['alt']
        busienss_dict['review-count'] = soup.find('div', attrs={'class':'biz-rating biz-rating-very-large clearfix'}).span.text
        get_page_reviews(soup, business_dict, database)
        # Find if there is a next button, if not end loop
        next_page = soup.find('a', attrs={'class': 'u-decoration-none next pagination-links_anchor'})
        if next_page:
            yelp_result_page = next_page['href']
        else:
            break

In [71]:
def get_page_reviews(soup, business_dict, database=None):
    result_list = soup.findAll('div', attrs={'class': 'review-content'})
    for result in result_list:
        review_details = {}
        review_details.update(business_dict)
        review_details['city'] = city
        review_details['text'] = result.p.text
        review_details['date'] = result.span.text
        review_details['rating'] = result.img['alt']
    database.ins(review_details)

In [56]:
crawl_results(search_term, city)

Scraping page: Best stripper clubsin Portland, OR
{'date': u'9/19/2015', 'text': u'Good vibes and hot girls!Great music and sound system!Great place to get rid of some unwanted cash!!!-1 star for smelling like a pungent combination of fried food and farts.', 'rating': u'4.0 star rating'}
{'date': u'4/3/2016', 'text': u'I&#39;ve never been to a strip club before coming here, but I have a hard time imagining any other place with more talented women. I think the laws of gravity itself stop upon walking through the doors. Besides the extraordinary performances provided the drinks were stiff at a good price and the atmosphere was as welcoming as a dive bar can get, but I never felt weird being a new comer/non regular/woman myself. No pressure to buy lap dances or anything like that. Very impressed.', 'rating': u'5.0 star rating'}
{'date': u'5/8/2014', 'text': u'1) a great place to ease your way into the world of strip clubs2) super strong drinks3) friendly staff4) killer concept - You stand

KeyboardInterrupt: 

In [62]:
url = 'https://www.yelp.com/biz/devils-point-portland?osq=Stripper+Clubs'
soup = BeautifulSoup(urllib2.urlopen(url).read())
h1 = soup.find('div', attrs={'class':'biz-rating biz-rating-very-large clearfix'})

In [64]:
h1 = soup.find('div', attrs={'class':'biz-rating biz-rating-very-large clearfix'}).img['alt']

In [68]:
soup.find('div', attrs={'class':'biz-rating biz-rating-very-large clearfix'}).span.text

u'179 reviews'

In [ ]:
<div class="biz-rating biz-rating-very-large clearfix">
                



    <div class="i-stars i-stars--large-4-half rating-very-large" title="4.5 star rating">
        <img class="offscreen" height="303" src="https://s3-media2.fl.yelpcdn.com/assets/srv0/yelp_design_web/9b34e39ccbeb/assets/img/stars/stars.png" width="84" alt="4.5 star rating">
    </div>


                    <span class="review-count rating-qualifier">
            179 reviews
    </span>

        </div>
    
<span class="review-count rating-qualifier">
            179 reviews
    </span>